<a href="https://colab.research.google.com/github/IlianKukov/Advanced-Machine-Learning-and-AI/blob/main/atiml23_lec07_lab07_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практически пример за разпределяне на продукти в класове чрез kNN

В нашият пример ще използваме данни за продукти от онлайн магазин, като имаме следните колони:

    uniq_id - уникалният идентификатор на продукта
    product_name - наименованието на продукта
    manufacturer - фирмата, която е изработила продукта
    price - единичната цена
    number_available_in_stock - колко броя има на склад
    number_of_reviews - брой мнения от клиентите
    number_of_answered_questions - брой отговорени въпроси относно продукта
    average_review_rating - средна оценка на продукта
    popularity - колко е популярен сред клиентите

На базата на тези данни, когато има нов запис, ще е необходимо да можем да преценим колко популярен е продуктът, като чрез алгоритъма kNN го класифицираме в някоя от трите категории за популярност - ниска, средна или висока.
## Стъпка 1: Импорт на библиотеки и зареждане на данните

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

In [ ]:

#df = pd.read_csv('./datasets/products-final.csv', sep=',')
df = pd.read_csv('./amazon_co-ecommerce_sample.csv', sep=',') 
df.head()

FileNotFoundError: ignored

In [ ]:
df.manufacturer = df.manufacturer.astype(str)
del df['price'] 
del df['number_available_in_stock'] 
#df[pd.to_numeric(df['number_available_in_stock'], errors='coerce').notnull()]
df[pd.to_numeric(df['number_of_reviews'], errors='coerce').notnull()]

## Стъпка 2: LabelEncoder

Използваме LabelEncoder, за да преобразуваме качествените данни в количествени.

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()
for column in df.columns:
    print (column )
          
       
    df[column] = labelencoder.fit_transform(df[column].astype(str))
 
df.head()

## Стъпка 3: Разделяне на данните на обучаваща и тестова извадка

Заделяме 70% от данните за обучението и 30% за тест.

In [ ]:
from sklearn.model_selection import train_test_split

X=df.drop(['number_of_reviews'], axis=1)
Y=df['number_of_reviews']
 
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.3,
                                                    random_state=17)

In [ ]:
## Стъпка 4: Обучение на модела

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)

#%%time
knn.fit(X_train, y_train)


In [ ]:
## Стъпка 5: Проверка на качеството на модела

In [ ]:
from sklearn.metrics import accuracy_score

knn_pred = knn.predict(X_test)
 
accuracy_score(y_test, knn_pred)

In [ ]:
## Стъпка 6: Избор на оптимални параметри

In [ ]:

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


knn_pipe = Pipeline([('scaler', StandardScaler()),
                     ('knn', KNeighborsClassifier(n_jobs=3))])
 
knn_params = {'knn__n_neighbors':range(1,10)}


knn_grid = GridSearchCV(knn_pipe, knn_params,cv=5, n_jobs=3, verbose=True)

#%%time
knn_grid.fit(X_train, y_train)

knn_grid.best_params_, knn_grid.best_score_

In [ ]:

accuracy_score(y_test, knn_grid.predict(X_test))